<a href="https://colab.research.google.com/github/Vicky-0222/NLP/blob/master/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Классические методы преобразования текста**

### Установка необходимых библиотек

In [1]:
!pip install pymorphy3

### Импорт библиотек и данных

In [2]:
import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import math
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
texts = ['Сегодня я решил пойти в парк. Парк был полон людей, и я наслаждался атмосферой.',
         'Сегодня я выбрался в парк, чтобы насладиться свежим воздухом. Парк был наполнен смехом и радостью, и я чувствовал себя прекрасно.',
         'Сегодня я решил провести время в парке. Парк был оживлённым и ярким, и я наслаждался каждым моментом.']

### Применить к текстам лемматизацию, удаления стоп слов и токенизацию по словам

In [4]:
morph = pymorphy3.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

def preprocessing(texts: list[str]) -> list[list[str]]:
  preprocessing_texts = []
  for text in texts:
    # токенизация
    tokens = word_tokenize(text.lower())
    # лемматизация и удаление стоп слов
    lemmatized_words = [
      morph.parse(token)[0].normal_form
      for token in tokens
      if token not in stop_words]

    preprocessing_texts.append(lemmatized_words)
  return preprocessing_texts

In [5]:
preprocessed_texts = preprocessing(texts)
print(preprocessed_texts)

[['сегодня', 'решить', 'пойти', 'парк', '.', 'парк', 'полный', 'человек', ',', 'наслаждаться', 'атмосфера', '.'], ['сегодня', 'выбраться', 'парк', ',', 'насладиться', 'свежий', 'воздух', '.', 'парк', 'наполнить', 'смех', 'радость', ',', 'чувствовать', 'прекрасно', '.'], ['сегодня', 'решить', 'провести', 'время', 'парка', '.', 'парк', 'оживлённый', 'яркий', ',', 'наслаждаться', 'каждый', 'момент', '.']]


### Реализовать Bag of Words

In [6]:
def make_dict(texts: list[str]) -> dict[str]:
  word_freq = {}

  for i, words in enumerate(preprocessed_texts):
      for word in words:
          if word not in word_freq:
              word_freq[word] = {}
          if f'text_{i}' not in word_freq[word]:
              word_freq[word][f'text_{i}'] = 0
          word_freq[word][f'text_{i}'] += 1

  return word_freq

In [7]:
BoW = make_dict(texts)
print(BoW)

{'сегодня': {'text_0': 1, 'text_1': 1, 'text_2': 1}, 'решить': {'text_0': 1, 'text_2': 1}, 'пойти': {'text_0': 1}, 'парк': {'text_0': 2, 'text_1': 2, 'text_2': 1}, '.': {'text_0': 2, 'text_1': 2, 'text_2': 2}, 'полный': {'text_0': 1}, 'человек': {'text_0': 1}, ',': {'text_0': 1, 'text_1': 2, 'text_2': 1}, 'наслаждаться': {'text_0': 1, 'text_2': 1}, 'атмосфера': {'text_0': 1}, 'выбраться': {'text_1': 1}, 'насладиться': {'text_1': 1}, 'свежий': {'text_1': 1}, 'воздух': {'text_1': 1}, 'наполнить': {'text_1': 1}, 'смех': {'text_1': 1}, 'радость': {'text_1': 1}, 'чувствовать': {'text_1': 1}, 'прекрасно': {'text_1': 1}, 'провести': {'text_2': 1}, 'время': {'text_2': 1}, 'парка': {'text_2': 1}, 'оживлённый': {'text_2': 1}, 'яркий': {'text_2': 1}, 'каждый': {'text_2': 1}, 'момент': {'text_2': 1}}


### Реализовать TF-IDF

In [8]:
def count_num_words(texts: list[str]) -> list[int]:
    num_words = []
    for text in texts:
        num_words.append(len(word_tokenize(text)))
    return num_words

In [9]:
def compute_tf(BoW: dict[str], texts: list[str]) -> dict[str]:
    tf = {}
    num_words = count_num_words(texts)

    for word, freq in BoW.items():
        tf[word] = {}
        for doc, count in freq.items():
            doc_index = int(doc.split('_')[1])
            tf[word][doc] = count / num_words[doc_index]

    return tf

In [10]:
tf = compute_tf(BoW, texts)
print(tf)

{'сегодня': {'text_0': 0.058823529411764705, 'text_1': 0.041666666666666664, 'text_2': 0.05}, 'решить': {'text_0': 0.058823529411764705, 'text_2': 0.05}, 'пойти': {'text_0': 0.058823529411764705}, 'парк': {'text_0': 0.11764705882352941, 'text_1': 0.08333333333333333, 'text_2': 0.05}, '.': {'text_0': 0.11764705882352941, 'text_1': 0.08333333333333333, 'text_2': 0.1}, 'полный': {'text_0': 0.058823529411764705}, 'человек': {'text_0': 0.058823529411764705}, ',': {'text_0': 0.058823529411764705, 'text_1': 0.08333333333333333, 'text_2': 0.05}, 'наслаждаться': {'text_0': 0.058823529411764705, 'text_2': 0.05}, 'атмосфера': {'text_0': 0.058823529411764705}, 'выбраться': {'text_1': 0.041666666666666664}, 'насладиться': {'text_1': 0.041666666666666664}, 'свежий': {'text_1': 0.041666666666666664}, 'воздух': {'text_1': 0.041666666666666664}, 'наполнить': {'text_1': 0.041666666666666664}, 'смех': {'text_1': 0.041666666666666664}, 'радость': {'text_1': 0.041666666666666664}, 'чувствовать': {'text_1':

In [11]:
def compute_idf(BoW: dict[str], num_docs: int) -> dict[str]:
    idf = {}

    for word in BoW:
        idf[word] = math.log(num_docs / len(BoW[word]))
    return idf

In [12]:
idf = compute_idf(BoW, len(texts))
print(idf)

{'сегодня': 0.0, 'решить': 0.4054651081081644, 'пойти': 1.0986122886681098, 'парк': 0.0, '.': 0.0, 'полный': 1.0986122886681098, 'человек': 1.0986122886681098, ',': 0.0, 'наслаждаться': 0.4054651081081644, 'атмосфера': 1.0986122886681098, 'выбраться': 1.0986122886681098, 'насладиться': 1.0986122886681098, 'свежий': 1.0986122886681098, 'воздух': 1.0986122886681098, 'наполнить': 1.0986122886681098, 'смех': 1.0986122886681098, 'радость': 1.0986122886681098, 'чувствовать': 1.0986122886681098, 'прекрасно': 1.0986122886681098, 'провести': 1.0986122886681098, 'время': 1.0986122886681098, 'парка': 1.0986122886681098, 'оживлённый': 1.0986122886681098, 'яркий': 1.0986122886681098, 'каждый': 1.0986122886681098, 'момент': 1.0986122886681098}


In [13]:
def compute_tfidf(tf: dict[str], idf: dict[str]) -> dict[str]:
    tfidf = {}

    for word in tf:
        tfidf[word] = {}
        for doc in tf[word]:
            tfidf[word][doc] = tf[word][doc] * idf[word]

    return tfidf

In [14]:
tfidf = compute_tfidf(tf, idf)
print(tfidf)

{'сегодня': {'text_0': 0.0, 'text_1': 0.0, 'text_2': 0.0}, 'решить': {'text_0': 0.023850888712244965, 'text_2': 0.02027325540540822}, 'пойти': {'text_0': 0.06462425227459469}, 'парк': {'text_0': 0.0, 'text_1': 0.0, 'text_2': 0.0}, '.': {'text_0': 0.0, 'text_1': 0.0, 'text_2': 0.0}, 'полный': {'text_0': 0.06462425227459469}, 'человек': {'text_0': 0.06462425227459469}, ',': {'text_0': 0.0, 'text_1': 0.0, 'text_2': 0.0}, 'наслаждаться': {'text_0': 0.023850888712244965, 'text_2': 0.02027325540540822}, 'атмосфера': {'text_0': 0.06462425227459469}, 'выбраться': {'text_1': 0.0457755120278379}, 'насладиться': {'text_1': 0.0457755120278379}, 'свежий': {'text_1': 0.0457755120278379}, 'воздух': {'text_1': 0.0457755120278379}, 'наполнить': {'text_1': 0.0457755120278379}, 'смех': {'text_1': 0.0457755120278379}, 'радость': {'text_1': 0.0457755120278379}, 'чувствовать': {'text_1': 0.0457755120278379}, 'прекрасно': {'text_1': 0.0457755120278379}, 'провести': {'text_2': 0.054930614433405495}, 'время': 